In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
import mysql.connector
from datetime import datetime
import io

# MySQL connection configuration
db = mysql.connector.connect(
    host="localhost",
    user="root",  # Replace with your MySQL Workbench username
    password="11223344minmin",  # Replace with your MySQL Workbench password
    database="university"  # Replace with your database name
)

cursor = db.cursor()

# Initialize the video capture object
video_capture = cv2.VideoCapture(0)

# Lists to hold the face encodings and names
known_face_encodings = []
known_face_names = []

# Fetch images from the database
cursor.execute("SELECT student_id, image FROM students")
rows = cursor.fetchall()

for student_id, image_data in rows:
    # Convert binary image data to a format readable by face_recognition
    image = face_recognition.load_image_file(io.BytesIO(image_data))
    
    # Get the face encodings (assuming one face per image)
    face_encodings = face_recognition.face_encodings(image)
    if face_encodings:  # Ensure at least one face encoding is found
        known_face_encodings.append(face_encodings[0])
        known_face_names.append(student_id)  # Use student_id as the name

# Copy the known face names to track attendance
students = known_face_names.copy()

# Initialize variables for face recognition process
face_locations = []
face_encodings = []
face_names = []
s = True

# Get the current date
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")

# Create a CSV file to save attendance
with open(current_date + '.csv', 'w+', newline='') as f:
    lnwriter = csv.writer(f)

    while True:
        # Capture a single frame from the video
        _, frame = video_capture.read()

        # Resize the frame to speed up processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR (OpenCV) to RGB (face_recognition)
        rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])

        if s:
            # Detect faces in the frame
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            face_names = []

            for face_encoding in face_encodings:
                # Compare detected faces with known faces
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = ""
                face_distance = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distance)

                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                
                face_names.append(name)
                
                # Check if the recognized person is in the student list
                if name in students:
                    students.remove(name)
                    print(students)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name, current_time])
            
        # Display the video feed with recognition
        cv2.imshow("attendance system", frame)

        # Exit the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture object and close all windows

video_capture.release()
cv2.destroyAllWindows()
